In [1]:
t1 = '''
The tutorial was quite helpful in understanding the concept. I'm going to share some quick tips for better photography. Don't forget to turn on the notification bell to stay updated. I was really excited about the new gadget launch. The event was a huge success with a massive turnout. Explaining complex topics in a simple way is my goal. It took me a while to figure out the settings. I'll be sharing a step-by-step guide very soon. She's an influencer known for her fashion tips and lifestyle vlogs. My video production setup includes a high-quality camera and professional lighting. Collaborating with fellow creators can lead to fresh and exciting content ideas. I use social media analytics to track the performance of my posts. The key to building a loyal audience is consistency in content delivery. I'm scripting my next video on tips for better storytelling. Can you help me brainstorm ideas for my podcast episodes? The thumbnail for my latest video is eye-catching, don't you think? I've scheduled a live Q&A session with my subscribers this weekend. Let's bring everything together, and look at how you will use the reward model in the reinforcement learning process to update the LLM weights, and produce a human aligned model. Remember, you wend to start with a model that already has good performance on your task of interests. First, you'll pass a prompt from your prompt dataset. In this case, a dog is, to the instruct LLM, which then generates a completion, in this case a furry animal.  Next, you sent this completion, and the original prompt to the reward model as the prompt completion pair. The reward model evaluates the pair based on the human feedback it was trained on, and returns a reward value. A higher value such as 0.24 as shown here represents a more aligned response. A less aligned response would receive a lower value, such as negative 0.53. You'll then pass this reward value for the prom completion pair to the reinforcement learning algorithm to update the weights of the LLM, and move it towards generating more aligned, higher reward responses.  Let's call this intermediate version of the model the RL updated LLM. These series of steps together forms a single iteration of the RLHF process. These iterations continue for a given number of epics, similar to other types of fine tuning. Here you can see that the completion generated by the RL updated LLM receives a higher reward score, indicating that the updates to weights have resulted in a more aligned completion. If the process is working well, you'll see the reward improving after each iteration as the model produces text that is increasingly aligned with human preferences. You will continue this iterative process until your model is aligned based on some evaluation criteria. For example, reaching a threshold value for the helpfulness you defined. You can also define a maximum number of steps, for example, 20,000 as the stopping criteria. At this point, let's refer to the fine-tuned model as the human-aligned LLM. One detail we haven't discussed yet is the exact nature of the reinforcement learning algorithm. This is the algorithm that takes the output of the reward model and uses it to update the LLM model weights so that the reward score increases over time. There are several different algorithms that you can use for this part of the RLHF process.  A popular choice is proximal policy optimization or PPO for short. PPO is a pretty complicated algorithm, and you don't have to be familiar with all of the details to be able to make use of it. However, it can be a tricky algorithm to implement and understanding its inner workings in more detail can help you troubleshoot if you're having problems getting it to work. To explain how the PPO algorithm works in more detail, I invited my AWS colleague, Ek to give you a deeper dive on the technical details. This next video is optional and you should feel free to skip it, and move on to the reward hacking video. You won't need the information here to complete the quizzes or this week's lab. However, I encourage you to check out the details as RLHF is becoming increasingly importend to ensure that LLMs behave in a safe and aligned manner in deployment. Using trl you can run one of the most popular Deep RL algorithms, PPO, in a distributed manner or on a single device! We leverage accelerate from the Hugging Face ecosystem to make this possible, so that any user can scale up the experiments up to an interesting scale. Fine-tuning a language model with RL follows roughly the protocol detailed below. This requires having 2 copies of the original model; to avoid the active model deviating too much from its original behavior, distribution you need to compute the logits of the reference model at each optimization step. This adds a hard constraint on the optimization process as you need always at least two copies of the model per GPU device. If the model grows in size, it becomes more and more tricky to fit the setup on a single GPU. Low-rank Adaptation, or LoRA for short, is a parameter-efficient fine-tuning technique that falls into the re-parameterization category. Let's take a look at how it works. As a quick reminder, here's the diagram of the transformer architecture that you saw earlier in the course. The input prompt is turned into tokens, which are then converted to embedding vectors and passed into the encoder and/or decoder parts of the transformer. In both of these components, there are two kinds of neural networks; self-attention and feedforward networks. The weights of these networks are learned during pre-training. After the embedding vectors are created, they're fed into the self-attention layers where a series of weights are applied to calculate the attention scores. During full fine-tuning, every parameter in these layers is updated. LoRA is a strategy that reduces the number of parameters to be trained during fine-tuning by freezing all of the original model parameters and then injecting a pair of rank decomposition matrices alongside the original weights. The dimensions of the smaller matrices are set so that their product is a matrix with the same dimensions as the weights they're modifying. You then keep the original weights of the LLM frozen and train the smaller matrices using the same supervised learning process you saw earlier this week. For inference, the two low-rank matrices are multiplied together to create a matrix with the same dimensions as the frozen weights. You then add this to the original weights and replace them in the model with these updated values. You now have a LoRA fine-tuned model that can carry out your specific task. Because this model has the same number of parameters as the original, there is little to no impact on inference latency. Researchers have found that applying LoRA to just the self-attention layers of the model is often enough to fine-tune for a task and achieve performance gains. However, in principle, you can also use LoRA on other components like the feed-forward layers. But since most of the parameters of LLMs are in the attention layers, you get the biggest savings in trainable parameters by applying LoRA to these weights matrices. Let's look at a practical example using the transformer architecture described in the Attention is All You Need paper. The paper specifies that the transformer weights have dimensions of 512 by 64. This means that each weights matrix has 32,768 trainable parameters. If you use LoRA as a fine-tuning method with the rank equal to eight, you will instead train two small rank decomposition matrices whose small dimension is eight. This means that Matrix A will have dimensions of 8 by 64, resulting in 512 total parameters. Matrix B will have dimensions of 512 by 8, or 4,096 trainable parameters. By updating the weights of these new low-rank matrices instead of the original weights, you'll be training 4,608 parameters instead of 32,768 and 86% reduction. Because LoRA allows you to significendly reduce the number of trainable parameters, you can often perform this method of parameter efficient fine tuning with a single GPU and avoid the need for a distributed cluster of GPUs. Since the rank-decomposition matrices are small, you can fine-tune a different set for each task and then switch them out at inference time by updating the weights. Suppose you train a pair of LoRA matrices for a specific task; let's call it Task A. To carry out inference on this task, you would multiply these matrices together and then add the resulting matrix to the original frozen weights. You then take this new summed weights matrix and replace the original weights where they appear in your model. You can then use this model to carry out inference on Task A. If instead, you wend to carry out a different task, say Task B, you simply take the LoRA matrices you trained for this task, calculate their product, and then add this matrix to the original weights and update the model again. The memory required to store these LoRA matrices is very small. So in principle, you can use LoRA to train for many tasks. Switch out the weights when you need to use them, and avoid having to store multiple full-size versions of the LLM. How good are these models? This data is pulled from many sources, including scrapes off the Internet and corpora of texts that have been assembled specifically for training language models. In this self-supervised learning step, the model internalizes the patterns and structures present in the language. These patterns then enable the model to complete its training objective, which depends on the architecture of the model, as you'll see shortly. During pre-training, the model weights get updated to minimize the loss of the training objective. The encoder generates an embedding or vector representation for each token. Pre-training also requires a large amount of compute and the use of GPUs. Note, when you scrape training data from public sites such as the Internet, you often need to process the data to increase quality, address bias, and remove other harmful content. As a result of this data quality curation, often only 1-3% of tokens are used for pre-training. You should consider this when you estimate how much data you need to collect if you decide to pre-train your own model. Earlier this week, you saw that there were three variance of the transformer model; encoder-only encoder-decoder models, and decode-only. Each of these is trained on a different objective, and so learns how to carry out different tasks. Encoder-only models are also known as Autoencoding models, and they are pre-trained using masked language modeling. Here, tokens in the input sequence or randomly mask, and the training objective is to predict the mask tokens in order to reconstruct the original sentence. This is also called a denoising objective. Autoencoding models spilled bi-directional representations of the input sequence, meaning that the model has an understanding of the full context of a token and not just of the words that come before. Encoder-only models are ideally suited to task that benefit from this bi-directional contexts. You can use them to carry out sentence classification tasks, for example, sentiment analysis or token-level tasks like named entity recogpolicyon or word classification. Some well-known examples of an autoencoder model are BERT and RoBERTa. Now, let's take a look at decoder-only or autoregressive models, which are pre-trained using causal language modeling. Here, the training objective is to predict the next token based on the previous sequence of tokens. Predicting the next token is sometimes called full language modeling by researchers. Decoder-based autoregressive models, mask the input sequence and can only see the input tokens leading up to the token in question. The model has no knowledge of the end of the sentence. The model then iterates over the input sequence one by one to predict the following token. A useful example to illustrate these ideas is training a model to play Tic-Tac-Toe. Let's take a look. In this example, the agent is a model or policy acting as a Tic-Tac-Toe player. Its objective is to win the game. The environment is the three by three game board, and the state at any moment, is the current configuration of the board. The action space comprises all the possible positions a player can choose based on the current board state. The agent makes decisions by following a strategy known as the RL policy. Now, as the agent takes actions, it collects rewards based on the actions' effectiveness in progressing towards a win. The goal of reinforcement learning is for the agent to learn the optimal policy for a given environment that maximizes their rewards. This learning process is iterative and involves trial and error. Ipolicyally, the agent takes a random action which leads to a new state. From this state, the agent proceeds to explore subsequent states through further actions. The series of actions and corresponding states form a playout, often called a rollout. As the agent accumulates experience, it gradually uncovers actions that yield the highest long-term rewards, ultimately leading to success in the game. Now let's take a look at how the Tic-Tac-Toe example can be extended to the case of fine-tuning large language models with RLHF. In this case, the agent's policy that guides the actions is the LLM, and its objective is to generate text that is perceived as being aligned with the human preferences. 
'''

t2 = '''
Tutorial ne concept samajhne mein kaafi madad ki. Main kuch tezi se photography ke behtar tips share karne ja raha hoon. Notification bell ko on kar lena mat bhoolna taaki updated reh sakein. Naye gadget launch ke liye main sach mein excited tha. Event bada successful tha, bahut saare log aaye the. Complex topics ko simple tareeke se samjhana mera goal hai. Settings ko samajhne mein mujhe thoda samay lag gaya. Main bahut jald ek step-by-step guide share karunga. Woh ek influencer hai jinhein fashion tips aur lifestyle vlogs ke liye jaana jaata hai. Mere video production setup mein ek high-quality camera aur professional lighting shamil hai. Doosre creators ke saath collaboration karke naye aur exciting content ideas mil sakti hai. Main social media analytics ka istemal apne posts ke performance ko track karne ke liye karta hoon. Ek nishta audience banane ka mool mantra hai content delivery mein consistency. Main apne agle video ke script par kaam kar raha hoon jo better storytelling ke tips par hoga. Kya aap meri podcast episodes ke liye ideas brainstorm karne mein meri madad kar sakte hain? Meri latest video ka thumbnail eye-catching hai, kya aapko nahi lagta? Mainne is weekend par apne subscribers ke saath live Q&A session ka schedule kiya hai. Chaliye sab kuch ek saath lete hain, aur dekhte hain ki aap kaise reward model ka upyog karke reinforcement learning process mein LLM ke weights ko update karenge, aur ek human aligned model utpann karenge. Yaad rakhiye, aapko apne task ke liye pehle se hi acchi performance wala model se shuru karna hai. Pehle, aap apne prompt dataset se ek prompt ko pass karenge. Is case mein, ek kutta hai, LLM ko instruct karenge, jo phir ek puri completion generate karta hai, is case mein ek furry animal. Agla, aap yeh completion aur asli prompt ko reward model ko bhejenge jaise prompt completion pair. Reward model pair ko evaluate karta hai insani feedback ke adhar par jis par usne training liya tha, aur ek reward value lautaata hai. Ek jyada value jaise 0.24 jo yahaan dikhaya gaya hai, ek zyada aligned response ko darshata hai. Kam aligned response ko ek kam value jaise ki negative 0.53 milta hai. Fir aap yeh reward value ko prompt completion pair ke liye reinforcement learning algorithm ko bhejenge LLM ke weights ko update karne ke liye, aur use generate karne ke liye zyada aligned, zyada reward responses. Chaliye is beech ka version ko RL updated LLM bula lete hain. Ye series of steps ek RLHF process ka single iteration banati hain. Ye iterations ek diye gaye number of epochs ke liye continue hoti hain, doosre fine-tuning ke prakar ke tarah. Yahaan dekha ja sakta hai ki RL updated LLM dwara generate ki gayi completion ko ek zyada reward score mila hai, iska matlab hai ki weights ke updates se ek zyada aligned completion aayi hai. Agar process acche se kaam kar rahi hai, toh aap har iteration ke baad reward mein sudhar dekhenge jab model text utpann karta hai jo insani pasand ke saath badhta hai. Aap is iterative process ko aage badhate rahenge jab tak aapka model kisi moolyaankan criteria ke adhar par aligned ho jaata hai. For example, aapne helpfulness ke liye ek drekht value ko paane tak. Aap ek adhikansh sankhya ke kadam ka niyamit kar sakte hain, jaise ki 20,000, jaise ki rukavat ke maamle mein. Is samay, chaliye fine-tuned model ko human-aligned LLM ke roop mein refer karein. Ek cheez humne abhi tak nahi charcha ki hai, woh hai reinforcement learning algorithm ka vyakti swaroop. Ye algorithm reward model ka output leta hai aur iska istemal LLM ke weights ko update karne mein karta hai taaki reward score samay ke saath badhe. Is RLHF process ka is hisse ke liye aap kai alag algorithms ka istemal kar sakte hain. Ek lokpriya chunav hai proximal policy optimization ya PPO ke liye. PPO ek kaafi complicated algorithm hai, aur aapko iske sabhi tafseelat se parichit hone ki zaroorat nahi hai ise istemal karne ke liye. Lekin, agar aapko iskaam ko karne mein koi samasya aa rahi hai, toh iske andar kaam kaise karta hai ko samajhna aapki madad kar sakta hai. PPO algorithm kaam kaise karta hai ko aur bhi vistaar se samjhane ke liye, maine apne AWS saathi, Ek ko bulaya hai aapko technical tafseelat mein gahraee se samjhane ke liye. Ye agla video optional hai aur aap isko chhodkar seedhe reward hacking video par jaa sakte hain. Aapko quizzes ya is haft ke lab ko poora karne ke liye yahaan di gayi jaankari ki zaroorat nahi hogi. Lekin, main aapko protsahan deta hoon ki aap tafseelat ko dekhein kyunki RLHF ki bhoomika badh rahi hai taaki LLMs deployment mein ek surakshit aur aligned tareeke se vyavhaar karein. trl ka istemal karke aap ek prasiddh Deep RL algorithms, PPO ko ek vitarit tareeke se ya ek single device par chala sakte hain! Hum Hugging Face ecosystem se accelerate ka istemal karte hain taaki koi bhi upyogkarta eksperiments ko ek rochak paimane tak scale kar sakein. Ek language model ko RL ke saath fine-tune karna lagbhag neeche vistarit protocol ko follow karna shuru karta hai. Iska matlab hai ki aapke paas asli model ke 2 nukkaron ki kopiya honi chahiye; asli vyavhaar se bahut zyada bhatakne se bachne ke liye, aapko har optimization kadam par reference model ke logits ko compute karna hoga. Ye ek kathin constraint ko optimization process par dalta hai kyun ki hamesha kam se kam do model ki kopiya ek GPU device ke liye chahiye hoti hai. Agar model ka size badh jaata hai, toh ek GPU par setup fit karna aur bhi zyada mushkil ho jaata hai. Low-rank Adaptation, ya LoRA in short, ek parameter-efficient fine-tuning technique hai jo re-parameterization category mein aata hai. Chaliye dekhte hain ki ye kaise kaam karta hai. Ek tezi se yaad dilane ke liye, yahaan pehle se padha hua transformer architecture ka diagram hai jo aapne is course mein pehle dekha tha. Input prompt ko tokens mein badal diya jaata hai, jo phir embedding vectors mein convert kiye jaate hain aur transformer ke encoder aur/va decoder hisson mein pass kiye jaate hain. In dono components mein, do prakar ke neural networks hote hain; self-attention aur feedforward networks. In networks ke weights pre-training ke dauran sikhe jaate hain. Embedding vectors banane ke baad, unhe self-attention layers mein feed kiya jaata hai jahan ek series of weights ka istemal attention scores ka calculation karne ke liye kiya jaata hai. Poora fine-tuning ke dauran, in layers ke har parameter ko update kiya jaata hai. LoRA ek strategy hai jo fine-tuning ke dauran train ki jaane waali parameters ki sankhya ko kam kar deta hai jo ki original model ke saath freeze kiye gaye hote hain aur phir original weights ke saath ek pair of rank decomposition matrices ko inject karta hai. Chhote matrices ke dimensions itne set kiye jaate hain ki unka product unke modify kiye gaye weights ke samaan dimensions wala ek matrix ho. Fir aap LLM ke original weights ko freeze karte hain aur in chhoti matrices ko wahi supervised learning process se train karte hain jo aapne is hafte pehle dekha tha. Inference ke liye, do low-rank matrices ko ek dusre se multiply kiya jaata hai ek matrix utpann karne ke liye jo freeze ki gayi weights ke samaan dimensions wala hota hai. Fir aap isko original weights mein jodte hain aur model mein in naye updated values ko daalte hain. Ab aapke paas ek LoRA fine-tuned model hai jo aapki khaas task ko kar sakta hai. Kyunki is model mein original ke samaan sankhya ke parameters hote hain, iska inference latency par koi asar nahi hota hai. Researchers ne paya hai ki LoRA ko model ke sirf self-attention layers par lagane se aksar task ke liye fine-tune kar sakte hain aur performance ko badha sakte hain. Lekin, tatva mein, aap feed-forward layers jaise doosre components par bhi LoRA ka istemal kar sakte hain. Lekin kyunki LLMs ke adhikansh parameters self-attention layers mein hote hain, aap in weights matrices par LoRA ka istemal karke train karne se trainable parameters mein sabse badi bachat pa sakte hain. Chaliye ek vyavharik udaharan dekhte hain jo Attention is All You Need paper mein di gayi transformer architecture ka istemal karta hai. Paper deta hai ki transformer ke weights ke dimensions 512 by 64 hote hain. Iska matlab hai ki har weights matrix mein 32,768 trainable parameters hote hain. Agar aap LoRA ko ek fine-tuning method ke roop mein rank barabar aath ke saath istemal karte hain, toh aap do chhote rank decomposition matrices ko train karenge jinki chhoti dimension aath hoti hai. Iska matlab hai ki Matrix A ke dimensions 8 by 64 honge, jisme 512 total parameters honge. Matrix B ke dimensions 512 by 8 honge, yaani 4,096 trainable parameters honge. In naye low-rank matrices ke weights ko update karke original weights ke bajaye, aap 32,768 ke bajaye 4,608 parameters ko train karenge aur 86% reduction hoga. Kyunki LoRA aapko trainable parameters ki sankhya ko significendly kam karne ki anumati deta hai, aap is method ko ek single GPU ke saath karte hue aksar kar sakte hain aur ek distributed cluster of GPUs ki zaroorat se bach sakte hain. Kyunki rank-decomposition matrices chhote hote hain, aap alag alag task ke liye ek naye set ke liye fine-tune kar sakte hain aur phir inference ke samay inhein badal sakte hain weights ko update karke. Maan lijiye aapne kisi vishesh task ke liye ek pair of LoRA matrices ko train kiya; chaliye ise Task A bulate hain. Is task par inference ko carry out karne ke liye, aap in matrices ko multiply karenge aur phir is matrix ko original frozen weights mein jodenge. Fir aap is naye sums weights matrix ko original weights mein badalenge jahan yeh aapke model mein dikhai dete hain. Ab aap is model ka istemal Task A par inference karne ke liye kar sakte hain. Agar aapko thayd mein, koi alag task, maan lijiye Task B, carry out karna hai, toh aap bas Task ke liye train kiye gaye LoRA matrices ko lenge, unka product calculate karenge, aur phir is matrix ko original weights mein add karenge aur phir se model ko update karenge. In LoRA matrices ko store karne ke liye zaroori memory bahut kam hoti hai. Iska matlab hai ki tatva mein, aap LoRA ka istemal karke bahut saare tasks ke liye train kar sakte hain. Jab aap unhe istemal karne ki zaroorat ho, toh unhe weights ko badalne se, aur LLM ke multiple full-size versions ko store karne ki zaroorat se bach sakte hain. Ye models kitne acche hain? Ye data kayi sources se liya gaya hai, jismein internet se scrape ki gayi jaankariyon aur text corpora shaamil hain jo language models ko training ke liye specifically compile kiye gaye hain. Is self-supervised learning ke kadam mein, model bhasha mein mojood patterns aur structures ko internalize karta hai. Ye patterns phir model ko uske training objective ko poora karne mein madad karte hain, jo model ke architecture par nirbhar karta hai, jaise ki aap jald hi dekhenge. Pre-training ke dauran, model weights ko update kiya jaata hai training objective ke loss ko minimize karne ke liye. Encoder har token ke liye ek embedding ya vector representation banata hai. Pre-training ko bhi ek bada amount of compute aur GPUs ka upyog karna hota hai. Note karein, jab aap public sites jaise ki internet se training data scrape karte hain, toh aapko data ko process karne ki zaroorat hoti hai quality ko badhane ke liye, bias ko address karne ke liye, aur doosre harmful content ko remove karne ke liye. Is data quality curation ke natije mein, aksar sirf 1-3% tokens ka istemal pre-training ke liye hota hai. Aapko yeh dhyan mein rakhna chahiye jab aap apna khud ka model pre-train karne ka faisla karte hain. Is hafte pehle, aapne dekha ki transformer model ke teen prakar hain; sirf encoder, encoder-decoder models, aur decode-only. Har ek ka alag objective hai, aur isliye har ek alag task ko karne ka tarika seekhta hai. Sirf encoder wale models ko Autoencoding models bhi jaante hain, aur ye masked language modeling ka istemal karke pre-train hote hain. Yahaan, input sequence ke tokens ko ya toh randomly mask kiya jata hai, aur training objective mask tokens ko predict karne ka hota hai taki asli sentence ko reconstruct kiya ja sake. Ye bhi ek denoising objective ke roop mein jaana jaata hai. Autoencoding models input sequence ka bi-directional representations banaate hain, matlab ki model ko ek token ka poora context samajh aata hai aur sirf pehle aane waale shabdon ka nahi. Sirf encoder wale models ideally suited hote hain task ke liye jo is bi-directional context se faayda uthate hain. Aap inhe sentence classification tasks ke liye istemal kar sakte hain, jaise ki sentiment analysis ya token-level tasks jaise named entity recognition ya word classification. Kuch prasiddh udaharan ek autoencoder model ke hain BERT aur RoBERTa. Ab, chaliye dekhte hain decoder-only ya autoregressive models ko, jo causal language modeling ka istemal karke pre-train hote hain. Yahaan, training objective hai ki agle token ka prediction karna pehle ke tokens ke sequence ke adhar par. Agla token predict karna kabhi kabhi researchers dwaara full language modeling kehte hain. Decoder-based autoregressive models, input sequence ko mask karte hain aur sirf input tokens ko dekhte hain jo sawal mein hain. Model ko vaakya ke ant ke baare mein koi gyaan nahi hota. Model fir input sequence ke har ek token ko predict karne ke liye ek ek karke iterate karta hai. Ye ideas ko illustrate karne ke liye ek upyogi udaharan hai model ko Tic-Tac-Toe khelne ke liye train karna. Chaliye dekhte hain. Is udaharan mein, agent ek model ya policy hai jo Tic-Tac-Toe player ke roop mein kaam karta hai. Iska uddeshya game jeetna hota hai. Environment teen by teen game board hota hai, aur kisi bhi samay ka state, board ka current configuration hota hai. Action space mein player jo bhi possible positions choose kar sakta hai, uska vyavhaar sthiti ke adhar par hota hai. Agent RL policy ke naam se jaani jaati hai. Ab, jab agent actions leta hai, toh woh actions ki effectiveness ke adhar par rewards collect karta hai jo game jeetne ki taraf pragati ke liye hai. Reinforcement learning ka uddeshya hai ki agent ko ek diye gaye environment ke liye ek optimal policy sikhna hai jo unke rewards ko maximum banaata hai. Ye learning process iterative hota hai aur trial and error ke saath hota hai. Aam taur par, agent ek random action leta hai jo ek naya state tak le jaata hai. Is state se, agent agle actions ke zariye aane waale states ko explore karne ke liye aage badhta hai. Actions aur corresponding states ka silsila ek playout ya rollout banate hain, jise aksar rollout kaha jata hai. Jab agent anubhav jama karta hai, toh woh dheere dheere un actions ko khole jata hai jo sabse zyada lambi dhaar rewards dete hain, ant mein khel mein safalta tak pahunchate hue. Ab chaliye dekhte hain ki Tic-Tac-Toe udaharan ko kaise fine-tune kar sakte hain RLHF ke saath. Is case mein, agent ka policy jo actions ko guide karta hai woh LLM hai, aur uska uddeshya hai text generate karna jo insani preferences ke saath aligned hai.
'''

In [2]:
t1l = t1.split(". ")
t2l = t2.split(". ")

# print the length of the two lists
print(len(t1l), len(t2l))

124 123


In [4]:
for (i, j) in zip(t1l, t2l):
    print(i + ".")
    print(j + ".")
    print()


The tutorial was quite helpful in understanding the concept.

Tutorial ne concept samajhne mein kaafi madad ki.

I'm going to share some quick tips for better photography.
Main kuch tezi se photography ke behtar tips share karne ja raha hoon.

Don't forget to turn on the notification bell to stay updated.
Notification bell ko on kar lena mat bhoolna taaki updated reh sakein.

I was really excited about the new gadget launch.
Naye gadget launch ke liye main sach mein excited tha.

The event was a huge success with a massive turnout.
Event bada successful tha, bahut saare log aaye the.

Explaining complex topics in a simple way is my goal.
Complex topics ko simple tareeke se samjhana mera goal hai.

It took me a while to figure out the settings.
Settings ko samajhne mein mujhe thoda samay lag gaya.

I'll be sharing a step-by-step guide very soon.
Main bahut jald ek step-by-step guide share karunga.

She's an influencer known for her fashion tips and lifestyle vlogs.
Woh ek influencer ha